In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4

In [3]:
with open("x.txt", "r", encoding="UTF-8") as f:
    text = f.read()

chars = sorted(set(text))
vocab_size = len(chars)

In [4]:
str_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_str = { i:ch for i, ch in enumerate(chars) }
encode = lambda str: [str_to_int[ch] for ch in str]
decode = lambda ints: "".join([int_to_str[i] for i in ints])

data = torch.tensor(encode(text), dtype=torch.long)

In [5]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            return logits, None
        
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)

        return index

In [7]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, 500)[0].tolist())
print(generated_chars)


/S’éL4Ewr?C.mc;Ar4$cmMo]:3w4’é6JwÉKwDçRd6Zã–T‘J”8uzö$CE—?CAU”XWöO]$ygw74/0FvOF9pW”GçT$aX!OyRm(“ZJï—$Qá!:Lãt$–éJ;MVPK7g3mãÉL;nZ66nC4b[:cq8?k1e…
r2(:[–âé;)/!L$:lEEksS:Gç(zf—rçqyP(cö5NÉC’çm/!HV’AG12G7ïâö…(1ñBaI)m,—?&[zDSd.-!l3u
3b8..N54Vm—u5NNX]PJ!No’;ANfçL$VYu?CB6c(
4ddMu!jgALKã–yOï!JA5mCE2nZqírzuÉ&?“;0x…W9d?y/SfÉEifâsf9oMTTÉDtdMM56,m–rzyïWNÉ(ñz!JZ–(“.
,/Cm&J VRxñK&&/ññ“;A4OY;8Eñãuzuã”Uph;AN3uEY;Bxâ’xP4NNu3-v]-…–$ÉztNI…r3á:EY“3á5dzijãjAiíKãGBl,R7l&GçYv“8ã–-!v‘R…RbâJwn!qwãUñCUQL“ïp1Tf]nEju“wQkmöñA–


In [19]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for _ in range(max_iters):
    xb, yb = get_batch("train")

    logits, loss = model.forward(xb, yb)

    # Do not add previous gradients
    optimizer.zero_grad(set_to_none=True)

    # Backpropogate loss
    loss.backward()

    # Use the computed gradient to optimize model
    optimizer.step()

print(loss.item())

2.352281093597412


In [21]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, 500)[0].tolist())
print(generated_chars)


Yo wa inod s we tthaneree 1z flagrifiera pmest e Dlene Wheapecotsteaplas rthe blthetin hisulequllangskir an ouly, winthele oreveard bigy, sedin borom—Inct hinthiove tonchime atand yor vernour blisintetos neechorint—ffofir g ARFoulletiteend heoc ablengresh hele. yitisird He u wle iong M“ÉButipac. pe has aperett o Yon theftuad wrale. therirouro t. oure’stod tmasiofepantondearrofouror cthof O&Bemaserc wowkllin d tevid nlfr e tare coveel L‘” ath qupor ar keads vende athe bengnord frof.
52XOnghecoss 
